In [1]:
import re
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem.Fingerprints.FingerprintMols import FingerprintMol
from rdkit.DataStructs import FingerprintSimilarity
from rdkit import RDLogger
from rdkit.Chem import AllChem
import pickle
import random
import itertools
from tqdm.auto import tqdm
import sys

RDLogger.DisableLog('rdApp.*')

Задаем параметры

In [2]:
find = 'MNXM1103458' # синтезируемый метаболит
max_iter = 100 # максимальное число посещений вершин - метаболитов
# параметры для алгоритма близости с частичным перебором
K = 3 
L = 3
# параметр для полного перебора
M = 4
test = ['MNXM147721', 'MNXM1586', 'MNXM167794', 'MNXM19114', 'MNXM1140', 'MNXM819862', 
        'MNXM167202', 'MNXM78797', 'MNXM9384', 'MNXM512016'] # метаболиты для сравнения алгоритмов

Читаем базу данных метаболитов MetaNetX

In [3]:
library = {}
fd = open('chem_prop.tsv')
while True:
    line = fd.readline()
    if line:
        if line[0] != '#':
            data = line.split('\t')
            if data[0] not in library:
                library[ data[0] ] = data[1:]
            else:
                print( f'{data[0]} - already recorded' )
    else:
        break
fd.close()

В следующих ячейке происходит формирование графа метаболической сети, его мы сформировали и сохранили зарнее в файл G.pickle, это связано с большим временем формирования графа.

In [4]:
def meta_graph(read=True):
    '''
    если read==True, то мы читаем заранее сохраненный граф метаболической сети из файла 'G.pickle'
    в противном случае мы формируем его непосредственно (этот граф не будет ничем отличаться от графа при read==True)
    '''
    G = {}
    if read:
        with open('G.pickle', 'rb') as handle:
            G = pickle.load(handle)
    else:
        fd = open('reac_prop.tsv')
        while True:
            line = fd.readline()
            if line:
                if line[0] != '#' and line.startswith('MNXR'):
                    data = line.split('\t')
                    left, right = data[1].split('=')
                    reac_l = data[0] + '_l'
                    reac_r = data[0] + '_r'
                    left = [el[:-1] for el in re.findall(r'MNXM\d+@|WATER@', left)]
                    right = [el[:-1] for el in re.findall(r'MNXM\d+@|WATER@', right)]
                    if len(left) == 1 and len(right) == 1:
                        if left[0] == right[0]:
                            continue
                    f = 1
                    for m_r in right:
                        if library[m_r][-1] == '\n':
                            f = -1

                    for m_l in left:
                        if library[m_l][-1] == '\n':
                            f = -1
                    for m_l in left:
                        try:
                            cs = Chem.CanonSmiles(library[m_l][-1].strip())
                        except:
                            f = -1

                    for m_r in right:
                        try:
                            cs = Chem.CanonSmiles(library[m_r][-1].strip())
                        except:
                            f = -1

                    if f == -1:
                        continue

                    for m_l in left:
                        if not(m_l in left and m_l in right):
                            if m_l not in G.keys():
                                G[m_l] = {'type': 'met', 'v': [reac_l]}
                            else:
                                G[m_l]['v'].append(reac_l)

                    for m_r in right:
                        if not(m_r in left and m_r in right):
                            if m_r not in G.keys():
                                G[m_r] = {'type': 'met', 'v': [reac_r]}
                            else:
                                G[m_r]['v'].append(reac_r)
                    l_v = [el for el in left if el in G.keys()]
                    r_v = [el for el in right if el in G.keys()]
                    if r_v != []:
                        G[reac_l] = {'type': 'reac', 'v': r_v}

                    if l_v != []:
                        G[reac_r] = {'type': 'reac', 'v': l_v}
            else:
                break
        fd.close()
    return G

In [5]:
G = meta_graph()

In [6]:
# чтение и предобработка штамма
stam_df = pd.read_csv('EcoNSmilesGLCconnected.csv', sep=';', names=['name', 'smiles', 'id'], header=None)
stam_ids = set(stam_df['id'].unique())
stam_ids.pop()
stam_smiles = set(stam_df['smiles'].unique())
stam_smiles.pop()
stam = []
for el in G.keys():
    if G[el]['type'] == 'met' and (el in stam_ids or library[el][-1].strip() in stam_smiles):
        stam.append(el)
stam = set(stam)

In [7]:
def dist(sm1, sm2):
    '''
    этой функции подается на вход две строки - SMILES метаболитов,
    она возвращает близость от 0 до 1 между метаболитами на основе алгоритма fingerprints и коэф. Танимото
    '''
    
    return FingerprintSimilarity(FingerprintMol(Chem.MolFromSmiles(sm1)), FingerprintMol(Chem.MolFromSmiles(sm2)))

In [8]:
def process(met):
    '''
    функция на вход получает метаболит,
    а возвращает реакцию с максимальным средним расстоянием до штамма
    '''
    
    mean = []
    for reac in G[met]['v']:
        s = 0
        k = 0
        for chld in G[reac]['v']:
            s += G[chld]['max_dist'][0]
            k += 1
        mean.append((s/k, reac))
    mean.sort(key=lambda x: x[0], reverse=True)
    return mean[0]

In [9]:
def find_path(met):
    '''
    функция принимает на вход метаболит met и рекурсивно строит метаболическое дерево, пока оно
    не станет решением задачи поиска метаболического пути или число посещенных вершин-метаболитов не
    превзойдет max_iter
    
    если flag == True, то значит решение найдено и в result будет лежать дерево в виде словаря вершин,
    где у каждой вершины будет поле родитель. При этом если по словарю result построить дерево, то не все его
    листья будут с пометкой leaf==True(это значит, что листовой метаболит в штамме), т.к. если мы уже встречали
    какой-то метаболит в графе, то при повторной встрече этого метаболита мы не будем дублировать для него
    рекурсивный обход графа.
    
    поэтому для успешной работы функции нужны глобальные переменные:
    result = {}
    n_iter = 0
    flag = True
    used = []
    '''
    
    global n_iter
    global flag
    if n_iter > max_iter:
        flag = False
        return -1
    n_iter += 1
    reac = process(met)[1]
    result[reac] =  {'parent': met, 'chld': G[reac]['v'], 'type': 'reac'} 
    if reac in used:
        return 1
    used.append(reac)
    
    for chld in G[reac]['v']:
        if chld in used:
            continue
        used.append(chld)
        if chld in stam:
            result[chld] = {'parent': reac, 'type': 'met', 'leaf': True}
        else:
            result[chld] =  {'parent': reac, 'type': 'met', 'leaf': False}
            find_path(chld)

In [10]:
def init_path_l(met, l):
    '''
    рекурсивная функция для перебора, которая заполняет массив max_l
    максимальным числом реакций для метаболитов на расстоянии l от синтезируемого метаболита
    '''
    
    if l > M:
        return -1
    max_l[l].append(len(G[met]['v']))
    for reac in G[met]['v']:
        if reac in used:
            continue
        used.append(reac)
        for chld in G[reac]['v']:
            if chld in used:
                continue
            used.append(chld)
            init_path_l(chld, l + 1)

In [11]:
def process_perm(met, l, perm):
    '''
    функция на вход получает метаболит met и возвращает реакцию в порядке перебора,
    определямым вектором perm и индексом l (если l >= M), в противном случае возвращает случайную реакцию
    '''
    
    if l >= M:
        return G[met]['v'][random.randint(0, len(G[met]['v']) - 1)]
    return G[met]['v'][min(perm[l], len(G[met]['v']) - 1)] 

In [12]:
def find_path_perm(met, l, perm):
    '''
    эта функция аналогична функции find_path_perm с тем отличием, что ей нужны
    массив perm(определяет порядок обхода) и индекс l (текущая глубина обхода)
    и у нее используется функция выбора реакции process_perm
    '''
    
    global n_iter
    global flag
    if n_iter > max_iter:
        flag = False
        return -1
    n_iter += 1

    reac = process_perm(met, l, perm)
    if reac in used:
        return 1
    used.append(reac)
    result[reac] =  {'parent': met, 'chld': G[reac]['v'], 'type': 'reac'} 
    for chld in G[reac]['v']:
        if chld in used:
            continue
        used.append(chld)
        if chld in stam:
            result[chld] = {'parent': reac, 'type': 'met', 'leaf': True}
        else:
            result[chld] =  {'parent': reac, 'type': 'met', 'leaf': False}
            find_path_perm(chld, l + 1, perm)

In [13]:
def processN(met, l, perm):
    '''
    функция на вход получает метаболит met и возвращает в порядке перебора(определяемым индексом l и вектором perm)
    одну из первых L реакций с максимальной средней близостью до метаболита (если l < K)
    в противном случае возвращает реакцию с максимальной средней близостью
    '''
    
    mean = []
    for reac in G[met]['v']:
        s = 0
        k = 0
        for chld in G[reac]['v']:
            s += G[chld]['max_dist'][0]
            k += 1
        mean.append((s/k, reac))
    mean.sort(key=lambda x: x[0], reverse=True)
    if l >= K:
        return mean[0]
    return mean[min(perm[l], len(mean) - 1)]

In [14]:
def find_pathN(met, l, perm):
    '''
    эта функция аналогична функции find_path_perm с тем отличием, что ей нужны
    массив perm(определяет порядок обхода) и индекс l (текущая глубина обхода)
    и у нее используется функция выбора реакции processN
    '''
    
    global n_iter
    global flag
    if n_iter > max_iter:
        flag = False
        return -1
    n_iter += 1

    reac = processN(met, l, perm)[1]
    if reac in used:
        return 1
    used.append(reac)
    result[reac] =  {'parent': met, 'chld': G[reac]['v'], 'type': 'reac'} 
    for chld in G[reac]['v']:
        if chld in used:
            continue
        used.append(chld)
        if chld in stam:
            result[chld] = {'parent': reac, 'type': 'met', 'leaf': True}
        else:
            result[chld] =  {'parent': reac, 'type': 'met', 'leaf': False}
            find_pathN(chld, l + 1, perm)

In [15]:
# запуск алгоритма на основе близости для метаболита find
result = {}
n_iter = 0
flag = True
used = []
find_path(find)
result

{'MNXR190736_l': {'parent': 'MNXM1103458',
  'chld': ['MNXM728294', 'MNXM9'],
  'type': 'reac'},
 'MNXM728294': {'parent': 'MNXR190736_l', 'type': 'met', 'leaf': False},
 'MNXR102345_l': {'parent': 'MNXM728294',
  'chld': ['MNXM144', 'MNXM3', 'MNXM593'],
  'type': 'reac'},
 'MNXM144': {'parent': 'MNXR102345_l', 'type': 'met', 'leaf': True},
 'MNXM3': {'parent': 'MNXR102345_l', 'type': 'met', 'leaf': True},
 'MNXM593': {'parent': 'MNXR102345_l', 'type': 'met', 'leaf': True},
 'MNXM9': {'parent': 'MNXR190736_l', 'type': 'met', 'leaf': True}}

In [16]:
# запуск алгоритма перебора для метаболита find
max_l = {i: [] for i in range(M + 1)}
used = []
init_path_l(find, 0)
vec = [] # вектор для перебора всех вариантов обхода графа метаболичской сети на глубине не более M
for k in max_l.keys():
    vec.append(max(max_l[k]))
vec = vec[:-1]
for i in range(len(vec)):
    vec[i] = list(range(vec[i]))
m = 1e9
for perm in tqdm(itertools.product(*vec), file=sys.stdout):
    result = {}
    n_iter = 0
    flag = True
    used = []
    find_path_perm(find, 0, perm)
    s = 0
    for k in result.keys():
        if result[k]['type'] == 'reac':
            s += 1
    if flag == True and s < m:
        m = s

0it [00:00, ?it/s]

In [27]:
vec_dist = [list(range(K)) for i in range(L)] # вектор для перебора в алгоритме на основе близости с частичным перебором

In [28]:
# запуск алгоритма на основе близости с частичным перебором для метаболита find
m = 1e9
res_G = {}
for perm in tqdm(itertools.product(*vec_dist), file=sys.stdout):
    result = {}
    n_iter = 0
    flag = True
    used = []
    find_pathN(find, 0, perm)
    s = 0
    for k in result.keys():
        if result[k]['type'] == 'reac':
            s += 1
    if flag == True and s < m:
        m = s
        res_G = result

0it [00:00, ?it/s]

In [19]:
%%time
# алгоритм на основе близости для метаболитов из test
dist_test = []
step = 0
for t in test:
    result = {}
    n_iter = 0
    flag = True
    used = []
    find_path(t)
    s = 0
    m = 1e9
    for k in result.keys():
        if result[k]['type'] == 'reac':
            s += 1
    if flag == True:
        m = s
    dist_test.append(m)

CPU times: user 108 ms, sys: 6 µs, total: 108 ms
Wall time: 107 ms


In [25]:
%%time
# алгоритм на основе перебора для метаболитов из test
perm_test = []
step = 0
for t in test:
    step += 1
    print('step' , step)
    max_l = {i: [] for i in range(M + 1)}
    used = []
    init_path_l(t, 0)
    vec = []
    for k in max_l.keys():
        vec.append(max(max_l[k]))
    vec = vec[:-1]
    for i in range(len(vec)):
        vec[i] = list(range(vec[i]))
    m = 1e9
    for perm in tqdm(itertools.product(*vec), file=sys.stdout):
        result = {}
        n_iter = 0
        flag = True
        used = []
        find_path_perm(t, 0, perm)
        s = 0
        for k in result.keys():
            if result[k]['type'] == 'reac':
                s += 1
        if flag == True and s < m:
            m = s
    perm_test.append(m)
    

step 1


0it [00:00, ?it/s]

step 2


0it [00:00, ?it/s]

step 3


...

step 9


0it [00:00, ?it/s]

step 10


0it [00:00, ?it/s]

CPU times: user 28min 54s, sys: 1.79 s, total: 28min 56s
Wall time: 28min 53s


In [29]:
%%time
# алгоритм на основе близсоти с частичным перебором для метаболитов из test
dist_test_w_perm = []
for t in test:
    m = 1e9
    res_G = {}
    for perm in tqdm(itertools.product(*vec_dist), file=sys.stdout):
        result = {}
        n_iter = 0
        flag = True
        used = []
        find_pathN(t, 0, perm)
        s = 0
        for k in result.keys():
            if result[k]['type'] == 'reac':
                s += 1
        if flag == True and s < m:
            m = s
            res_G = result
    dist_test_w_perm.append(m)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

CPU times: user 3.37 s, sys: 8.01 ms, total: 3.38 s
Wall time: 3.36 s


In [30]:
print(f'Среднее число реакций для алгоритма на основе близости {np.array(dist_test).mean():.2f}')
print(f'Среднее число реакций для алгоритма на основе перебора {np.array(perm_test).mean():.2f}')
print(f'Среднее число реакций для алгоритма на основе близости с частичным перебором {np.array(dist_test_w_perm).mean():.2f}')

Среднее число реакций для алгоритма на основе близости 8.30
Среднее число реакций для алгоритма на основе перебора 4.00
Среднее число реакций для алгоритма на основе близости с частичным перебором 4.50
